In [31]:
import torch
import torch.nn as nn
from transformers import GPT2Model, GPT2Tokenizer, GPT2Config
class GLA(nn.Module):
    def __init__(self):
        super().__init__()
        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        for i in range(12):
           print(self.gpt2.h[i].attn)
        self.gpt2_layers = []
        for i in range(12):
            self.gpt2_layers.append(self.gpt2.h[i])
        print(self.gpt2_layers)
    def layers(self):
        return self.gpt2_layers

In [32]:
model = GLA()
layers = model.layers()

GPT2SdpaAttention(
  (c_attn): Conv1D()
  (c_proj): Conv1D()
  (attn_dropout): Dropout(p=0.1, inplace=False)
  (resid_dropout): Dropout(p=0.1, inplace=False)
)
GPT2SdpaAttention(
  (c_attn): Conv1D()
  (c_proj): Conv1D()
  (attn_dropout): Dropout(p=0.1, inplace=False)
  (resid_dropout): Dropout(p=0.1, inplace=False)
)
GPT2SdpaAttention(
  (c_attn): Conv1D()
  (c_proj): Conv1D()
  (attn_dropout): Dropout(p=0.1, inplace=False)
  (resid_dropout): Dropout(p=0.1, inplace=False)
)
GPT2SdpaAttention(
  (c_attn): Conv1D()
  (c_proj): Conv1D()
  (attn_dropout): Dropout(p=0.1, inplace=False)
  (resid_dropout): Dropout(p=0.1, inplace=False)
)
GPT2SdpaAttention(
  (c_attn): Conv1D()
  (c_proj): Conv1D()
  (attn_dropout): Dropout(p=0.1, inplace=False)
  (resid_dropout): Dropout(p=0.1, inplace=False)
)
GPT2SdpaAttention(
  (c_attn): Conv1D()
  (c_proj): Conv1D()
  (attn_dropout): Dropout(p=0.1, inplace=False)
  (resid_dropout): Dropout(p=0.1, inplace=False)
)
GPT2SdpaAttention(
  (c_attn): Conv1D()


In [39]:
layers[0].attn = CNN()

In [38]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        pass
    def forward(self):
        pass

In [42]:
type(layers[0].ln_1)

torch.nn.modules.normalization.LayerNorm